In [1]:
import pandas as pd
import json
from deep_translator import GoogleTranslator
from razdel import sentenize
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-small')

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

C:\Users\huawei\anaconda3\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\huawei\.cache\huggingface\hub\models--intfloat--multilingual-e5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

In [2]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [3]:
def scores_calc(input_texts, len_messages):
    batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

    outputs = model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

    # normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    scores = (embeddings[:len_messages] @ embeddings[len_messages:].T) * 100
    return scores

In [6]:
passage_1 = """🏦 Сбер

Какие у меня мысли по компании.

Эта компания самая дорогая в России, ее стоимость более 6 трлн руб. Конечно же для этого есть основание — она генерирует самую большую прибыль, по крайней мере в этом году прибыль может составить более 1,5 трлн руб., поэтому Сбер по 6 трлн уже и не выглядит так дорого. 

Компания платит достойные дивиденды, а так как на рынке сейчас "правят" физики, дивиденды в таких компаниях будет основополагающим в рыночной оценке. На данный момент я ожидаю, что Сбер заплатит более 33 руб. дивидендами, примерно 34-35 руб. Сбер будет платить около 12-14% дивидендами, сейчас это не кажется какой-то большой доходностью при ставке 16% и доходности безриска в ОФЗ-ПК около 16%, и долгосрочных ОФЗ-ПД около 12%, но когда ставку снизят до 8-9%, такая зафиксированная доходность будет выглядеть привлекательно. Тем не менее надо понимать, что и ОФЗ-ПД при снижении ставки могут вырасти и составить конкуренцию Сберу как минимум в 2024-2025 гг. 

Компанию однозначно стоит рассмотреть подробно в 2024 году, что думаете друзья?"""

passage_2 = """
🏦 Тинькофф

Тинькофф оценивается рынком примерно в 0,62 трлн руб., при этом прибыль по итогам 2023 года может составить более 75 млрд руб.

Обратите внимание, для компании стоимости такая оценка была бы справедливой, но Тинькофф — это все-таки компания роста, которая последние 5 лет наращивала прибыль в среднем на 30% (не считая 2022 год), кроме того, темпы роста в следующие 3 года по самым скромным ожиданиям будут более 10% годовых. 

Вместе с тем, компания несет в себе инфраструктурные риски, а также даже если Тинькофф возобновит дивидендные выплаты, то они будут не более 5% годовых. 

На самом деле сложно определиться между акциями Сбера и Тинькофф, но можно с высокой вероятностью утверждать, что для того, чтобы получить доходность выше, чем сейчас в облигациях, необходимо брать на себя дополнительные риски, и тут рисковым инвесторам могут подойти именно акции TCS из-за инфраструктурных рисков.

Что думаете друзья, кто лучше?
"""

tickers = ['TCS Тинькофф', 'SBER Сбербанк', "NVTK Новатэк"]

input_texts = [passage_1] + tickers

print(scores_calc(input_texts))

TypeError: scores_calc() missing 1 required positional argument: 'len_messages'

In [7]:
input_texts

['🏦 Сбер\n\nКакие у меня мысли по компании.\n\nЭта компания самая дорогая в России, ее стоимость более 6 трлн руб. Конечно же для этого есть основание — она генерирует самую большую прибыль, по крайней мере в этом году прибыль может составить более 1,5 трлн руб., поэтому Сбер по 6 трлн уже и не выглядит так дорого. \n\nКомпания платит достойные дивиденды, а так как на рынке сейчас "правят" физики, дивиденды в таких компаниях будет основополагающим в рыночной оценке. На данный момент я ожидаю, что Сбер заплатит более 33 руб. дивидендами, примерно 34-35 руб. Сбер будет платить около 12-14% дивидендами, сейчас это не кажется какой-то большой доходностью при ставке 16% и доходности безриска в ОФЗ-ПК около 16%, и долгосрочных ОФЗ-ПД около 12%, но когда ставку снизят до 8-9%, такая зафиксированная доходность будет выглядеть привлекательно. Тем не менее надо понимать, что и ОФЗ-ПД при снижении ставки могут вырасти и составить конкуренцию Сберу как минимум в 2024-2025 гг. \n\nКомпанию однознач

In [8]:
moex_data = pd.read_csv('moex_tickers.csv', on_bad_lines='skip', encoding='windows-1251')
moex_data = moex_data[~moex_data['TRADE_CODE'].isnull()]
moex_data = moex_data[['TRADE_CODE', 'EMITENT_FULL_NAME', 'INSTRUMENT_TYPE']]
moex_data = moex_data[moex_data['INSTRUMENT_TYPE'] == 'Акция обыкновенная']
moex_data = moex_data[['TRADE_CODE', 'EMITENT_FULL_NAME']]
moex_data['ticker_and_name'] = moex_data['TRADE_CODE'] + ' ' + moex_data['EMITENT_FULL_NAME']
moex_tickers = moex_data['ticker_and_name'].values.tolist()

In [18]:
def predict_ticker(messages, moex_tickers):
    input_texts = messages + moex_tickers
    pred_labels_matrix = scores_calc(input_texts, len(messages))
    pred_labels = pred_labels_matrix.argmax(axis=1)
    pred_tickers = [moex_tickers[l] for l in pred_labels]
    return pred_tickers

In [19]:
moex_tickers[20:40]

['MAGN Публичное акционерное общество "Магнитогорский металлургический комбинат"',
 'MTLR Публичное акционерное общество "Мечел"',
 'MTSS Публичное акционерное общество "Мобильные ТелеСистемы"',
 'MOEX Публичное акционерное общество "Московская Биржа ММВБ-РТС"',
 'LKOH Публичное акционерное общество "Нефтяная компания "ЛУКОЙЛ"',
 'BELU Публичное акционерное общество "НоваБев Групп"',
 'NLMK Публичное акционерное общество "Новолипецкий металлургический комбинат"',
 'PIKK Публичное акционерное общество "ПИК-специализированный застройщик"',
 'PLZL Публичное акционерное общество "Полюс"',
 'RTKM Публичное акционерное общество "Ростелеком"',
 'SBER Публичное акционерное общество "Сбербанк России"',
 'CHMF Публичное акционерное общество "Северсталь"',
 'SELG Публичное акционерное общество "Селигдар"',
 'SVCB Публичное акционерное общество "Совкомбанк"',
 'FLOT Публичное акционерное общество "Современный коммерческий флот"',
 'TGKA Публичное акционерное общество "Территориальная генерирующая 

In [22]:
predict_ticker([passage_2], moex_tickers)

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 67998720 bytes.

In [42]:


translator = GoogleTranslator(source='auto', target='en')

In [48]:
with open('result.json', 'r', encoding='utf-8') as f:
    tg_data = json.load(f)

In [52]:
channel_messages = tg_data['messages']

In [75]:
[text_chunk.get('text', '') if isinstance(text_chunk, dict) else text_chunk.strip() for text_chunk in channel_messages[-109]['text']]

['\u200b\u200b',
 'Продолжение',
 '',
 'Акции отдельных компаний. ',
 'Это один из самых рисковых инструментов на рынке в рублях, но почему-то практически 100% инвесторов используют этот инструмент независимо от инвестиционного горизонта, допустимых рисков, цели инвестирования и т.д. Кому подойдет покупка акций? Я бы не рекомендовала покупать их с горизонтом до 3 или даже 5 лет. Наша страна в инвестиционном мире сейчас обладает самыми большими рисками, и любой глобальный инвестор скажет, что больше всего рисков на фондовом рынке акций российских компаний. Прежде чем инвестировать в акции, это нужно осознать. Самые-самые большие риски именно в акциях российских компаний. Не стоит использовать такой инструмент, если вы копите на машину, квартиру, учебу, лечение, отдых и т.д. Также не стоит использовать этот инструмент с горизонтом до 3-5 лет (будет зависеть от надёжности компаний). Если вы решили всё-таки использовать данный инструмент, то поймите, что покупка отдельных компаний может бы

In [67]:
channel_messages[-102]

{'id': 5014,
 'type': 'message',
 'date': '2023-11-12T17:52:34',
 'date_unixtime': '1699800754',
 'edited': '2023-11-12T17:52:48',
 'edited_unixtime': '1699800768',
 'from': 'NataliaBaffetovna | Инвестиции',
 'from_id': 'channel1400938880',
 'text': ['Когда я буду покупать облигации, писала тут: ',
  {'type': 'link', 'text': 'https://t.me/NataliaBaff/4949'},
  ''],
 'text_entities': [{'type': 'plain',
   'text': 'Когда я буду покупать облигации, писала тут: '},
  {'type': 'link', 'text': 'https://t.me/NataliaBaff/4949'},
  {'type': 'plain', 'text': ''}]}